In [1]:
import numpy as np
import pprint
import sys
if "..\\" not in sys.path:
    sys.path.append("..\\") 
from lib.envs.gridworld import GridworldEnv

In [2]:
pp = pprint.PrettyPrinter(indent=2)
env = GridworldEnv()

In [3]:
# Taken from policy evaluation

def policy_eval(policy, env, discount_factor=1.0, theta=0.00001):
    """
    Evaluate a policy given an environment and a full description of the environment's dynamics.
    
    Args:
        policy: [S, A] shaped matrix representing the policy.
        env: OpenAI env. env.P represents the transition probabilities of the environment.
            env.P[s][a] is a (prob, next_state, reward, done) tuple.
        theta: We stop evaluation once our value function change is less than theta for all states.
        discount_factor: gamma discount factor.
    
    Returns:
        Vector of length env.nS representing the value function.
    """
    # Start with a random (all 0) value function
    V = np.zeros(env.nS)
    V_updated = np.zeros(env.nS)
    while True:
        for s in range(env.nS):
            for a in range(env.nA):
                tmp_total_reward = 0
                boostrap_value = 0
                for prob, next_state, reward, _ in env.P[s][a]:
                    tmp_total_reward += prob*reward
                    boostrap_value += prob*V[next_state]
                V_updated[s] += policy[s, a] * (tmp_total_reward + discount_factor*boostrap_value)
        if np.max(np.abs(V_updated - V)) < theta:
            break
        else:
            V = V_updated
            V_updated = np.zeros(env.nS)
    return np.array(V)

In [22]:
def policy_improvement(env, policy_eval_fn = policy_eval, discount_factor = 1.0):
    """
    Policy Improvement Algorithm. Iteratively evaluates and improves a policy
    until an optimal policy is found.
    
    Args:
        env: The OpenAI envrionment.
        policy_eval_fn: Policy Evaluation function that takes 3 arguments:
            policy, env, discount_factor.
        discount_factor: Lambda discount factor.
        
    Returns:
        A tuple (policy, V). 
        policy is the optimal policy, a matrix of shape [S, A] where each state s
        contains a valid probability distribution over actions.
        V is the value function for the optimal policy.
        
    """
    # Start with a random policy
    policy = np.ones([env.nS, env.nA])/env.nA
    policy_update = np.zeros([env.nS, env.nA])
    while True:
        V = policy_eval(policy, env, discount_factor=discount_factor, theta=0.00001)
        q_value = np.zeros([env.nS, env.nA])
        for s in range(env.nS):
            for a in range(env.nA):
                reward = 0
                v = 0
                for (prob, next_state, reward, done) in env.P[s][a]:
                    reward += prob*reward
                    v += prob*V[next_state]
                q_value[s,a] = reward + v
        selected_action = np.argmax(q_value, axis = 1).tolist()
        for row, i in enumerate(selected_action):
            policy_update[row, i] = 1
        if np.max(np.abs(policy_update - policy)) < 0.0001:
            return policy, V
        else:
            policy = policy_update
            policy_update = np.zeros([env.nS, env.nA])
        

In [23]:
policy, v = policy_improvement(env)
print("Policy Probability Distribution:")
print(policy)
print("")

print("Reshaped Grid Policy (0=up, 1=right, 2=down, 3=left):")
print(np.reshape(np.argmax(policy, axis=1), env.shape))
print("")

print("Value Function:")
print(v)
print("")

print("Reshaped Grid Value Function:")
print(v.reshape(env.shape))
print("")

Policy Probability Distribution:
[[ 1.  0.  0.  0.]
 [ 0.  0.  0.  1.]
 [ 0.  0.  0.  1.]
 [ 0.  0.  1.  0.]
 [ 1.  0.  0.  0.]
 [ 1.  0.  0.  0.]
 [ 1.  0.  0.  0.]
 [ 0.  0.  1.  0.]
 [ 1.  0.  0.  0.]
 [ 1.  0.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  0.  1.  0.]
 [ 1.  0.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 1.  0.  0.  0.]]

Reshaped Grid Policy (0=up, 1=right, 2=down, 3=left):
[[0 3 3 2]
 [0 0 0 2]
 [0 0 1 2]
 [0 1 1 0]]

Value Function:
[ 0. -1. -2. -3. -1. -2. -3. -2. -2. -3. -2. -1. -3. -2. -1.  0.]

Reshaped Grid Value Function:
[[ 0. -1. -2. -3.]
 [-1. -2. -3. -2.]
 [-2. -3. -2. -1.]
 [-3. -2. -1.  0.]]



In [24]:
# Test the value function
expected_v = np.array([ 0, -1, -2, -3, -1, -2, -3, -2, -2, -3, -2, -1, -3, -2, -1,  0])
np.testing.assert_array_almost_equal(v, expected_v, decimal=2)